# 「Python による経済・経営分析のための実践的データサイエンス」第3章 (pp.81-83)

ウィキペディアの情報をデーターベース化した、DBpedia.orgから情報を取得する方法を紹介する。  
まず、データベースを操作するための、Sparqlwrapperパッケージをインストールする。

In [1]:
!pip install sparqlwrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00


Wikipedia にあるコンピューターゲームメーカーの情報をまとめて取得する

In [2]:
#提供されたサンプルのままだと、以下のエラーが起きる。これは、SSL証明書が正しくないサイトとのこと
#URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)>
#SSL証明書が正しくないサイトに対してPythonでアクセスする方法を追加する。
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Sparqlwrappeパッケージをインポートして、DBpedia.orgのSparcl Endpoint上からデータを取得する。

from SPARQLWrapper import SPARQLWrapper

# 日本のコンピューターゲームメーカーの情報について、企業名と概要を取得する
# SPARQLWrapperのインスタンスを作成し、SPARQLendpointとして http://ja.dbpedia.org/sparqlを指定する。データの朱徳方法はJSONとする。
sparql2 = SPARQLWrapper(endpoint='http://ja.dbpedia.org/sparql', returnFormat='json')
# setQuerry内にクエリを指定する。
# PREFIXとしてDBpedia.orgに指定されたオントロジー構造を指定する。
# select文では、?nameおよび?sbstractを取得することを指定する。このとき重複した内容を除去するため、distinctを指定する。
# where構文内に取得するデータを指定する。?companyについて、dbpedia.org内で「Category:日本のコンピューターゲームメーカー・ブランド」の分類が
# 行われている企業群をまとめて取得することを宣言する。
#その企業名(?name)および、企業の概要情報(?abstract)を取得する。
sparql2.setQuery("""
    PREFIX dbpedia-owl:  <http://dbpedia.org/ontology/>
    select distinct ?name ?abstract where {
    ?company <http://dbpedia.org/ontology/wikiPageWikiLink> <http://ja.dbpedia.org/resource/Category:日本のコンピュータゲームメーカー・ブランド> .
    ?company rdfs:label ?name .
    ?company <http://dbpedia.org/ontology/abstract> ?abstract .
}
""")
results2 = sparql2.query().convert()

In [5]:
print(results2)

{'head': {'link': [], 'vars': ['name', 'abstract']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'name': {'type': 'literal', 'xml:lang': 'ja', 'value': '1-UPスタジオ'}, 'abstract': {'type': 'literal', 'xml:lang': 'ja', 'value': '1-UPスタジオ株式会社（ワン-アップスタジオ、英: 1-UP Studio Inc.）は、コンピューターゲームソフトウェア開発を行う日本の企業。旧商号は株式会社ブラウニーブラウン。'}}, {'name': {'type': 'literal', 'xml:lang': 'ja', 'value': 'Coly'}, 'abstract': {'type': 'literal', 'xml:lang': 'ja', 'value': '株式会社coly（コリー、英: coly Inc.）は、モバイルオンラインゲームの企画・開発・運営などを主な事業とする日本の企業。コンピュータエンターテインメント協会正会員。'}}, {'name': {'type': 'literal', 'xml:lang': 'ja', 'value': 'Comcept'}, 'abstract': {'type': 'literal', 'xml:lang': 'ja', 'value': '株式会社comcept（コンセプト）は、大阪府大阪市に拠点を置くゲームソフト開発会社。'}}, {'name': {'type': 'literal', 'xml:lang': 'ja', 'value': 'Craft Egg'}, 'abstract': {'type': 'literal', 'xml:lang': 'ja', 'value': '株式会社Craft Egg（クラフトエッグ、英: Craft Egg Inc.）はスマートフォンアプリの企画・開発・運営を事業内容とする日本の企業。'}}, {'name': {'type': 'literal', 'xml:lang': 'ja', 'value': 'Cy

結果を整形する  
for文を用いデータを整形する。Python上でdict形式でデータが保持されているので、企業名および概要を一行ごとに取り出す。

In [7]:
#結果を整形しPrint する
for result in results2["results"]["bindings"]:
    print(result["name"]["value"],";",result["abstract"]["value"])

1-UPスタジオ ; 1-UPスタジオ株式会社（ワン-アップスタジオ、英: 1-UP Studio Inc.）は、コンピューターゲームソフトウェア開発を行う日本の企業。旧商号は株式会社ブラウニーブラウン。
Coly ; 株式会社coly（コリー、英: coly Inc.）は、モバイルオンラインゲームの企画・開発・運営などを主な事業とする日本の企業。コンピュータエンターテインメント協会正会員。
Comcept ; 株式会社comcept（コンセプト）は、大阪府大阪市に拠点を置くゲームソフト開発会社。
Craft Egg ; 株式会社Craft Egg（クラフトエッグ、英: Craft Egg Inc.）はスマートフォンアプリの企画・開発・運営を事業内容とする日本の企業。
Cygames ; 株式会社Cygames（サイゲームス、英: Cygames, Inc.）は、サイバーエージェントグループのモバイル向けゲームアプリおよび家庭用ゲームソフト開発事業を主とする企業である。サイバーエージェントの連結子会社（設立時は100%出資の完全子会社）であり、また、DeNAの持分法適用関連会社に該当する。
D4エンタープライズ ; 株式会社D4エンタープライズ（ディーフォーエンタープライズ）はインターネットを利用したコンテンツ配信サービスの企画、開発、運営、販売を主な事業内容とする日本の企業。
DETUNE ; 株式会社DETUNE（デチューン）は東京都杉並区に所在する音楽制作会社である。社長はナムコ（後のバンダイナムコゲームス）出身の佐野信義。
DONUTS (企業) ; 株式会社DONUTS（ドーナツ、登記上商号：株式会社Donuts）は、東京都渋谷区に本社を置く、クラウドサービスおよび動画・ライブ配信サービス、スマートフォンゲーム開発などを主体とする企業である。
Daisy2 ; Daisy2（でいじーでいじー）は、女性向けゲーム（乙女ゲーム）を制作するブランド。
Dragami Games ; 株式会社Dragami Games（ドラガミゲームス、英: Dragami Games, LTD.）は、日本のゲーム会社。株式会社エクストリームの連結子会社。
EXNOA ; 合同会社EXNOA（エクスノア、英: EXNOA LLC）は、ゲームプラットフォーム「DMM GAMES」の

次の例として、アメリカ合衆国のコンピューターゲームメーカーの情報を取得する

In [ ]:
from SPARQLWrapper import SPARQLWrapper

#アメリカ合衆国のコンピューターゲームメーカーの情報を取得する
sparql3 = SPARQLWrapper(endpoint='http://ja.dbpedia.org/sparql', returnFormat='json')
sparql3.setQuery("""
    PREFIX dbpedia-owl:  <http://dbpedia.org/ontology/>
    select distinct ?name ?abstract where {
    ?company <http://dbpedia.org/ontology/wikiPageWikiLink> <http://ja.dbpedia.org/resource/Category:アメリカ合衆国のコンピュータゲームメーカー・ブランド> .
    ?company rdfs:label ?name .
    ?company <http://dbpedia.org/ontology/abstract> ?abstract .
}
""")
results3 = sparql3.query().convert()

In [ ]:
#結果を整形しPrint する
for result in results3["results"]["bindings"]:
    print(result["name"]["value"],";",result["abstract"]["value"])

17-BIT ; 17-BITは、コンピューターゲームの制作会社。 ユナイテッド・ゲーム・アーティスツ、エレクトロニック・アーツなどで働いていたジェイク・カズダルが2009年にシアトルで設立した。スタジオをシアトルと京都に置く。レトロさにAIや物理演算などをプラスし、ジャンルにとらわれない作風を指向する。作品にはen:Skulls of the Shogun、en:Galak-Z: The Dimensionalがある。
2K Games ; 2K Gamesは、テイクツー・インタラクティブの子会社で、テレビゲームの販売を行う企業である。テイクツー・インタラクティブがセガから2,400万ドルでとVisual Conceptsを買収し、2005年1月25日に設立された。 2K Gamesという名前は、Visual Conceptsのゲームである2K seriesを由来としている。現在カリフォルニア州ノバトに本社を置く。
2K Marin ; 2K Marin はアメリカ合衆国カリフォルニア州ノバトとオーストラリアキャンベラに拠点を置くアメリカ合衆国とオーストラリアのコンピュータゲーム開発会社である。プレスリリースによると、スタジオの主な焦点は新規IPとほかの2K Studioとの共同開発製品の開発によせられている。 2K Marinは、ノバトスタジオBioshockのリリースの後にIrrational Games(短期間2K Boston/2K Australiaという名前であった)から派生した。2K Australia、つまりキャンベラスタジオはもともとIrrational Gamesの一部だった。新たなのゲームの発表に際して、2010年に2K Marinの一部となった。しかし、2K Marinの業績悪化のため、2K Australiaは、2011年に再び独立企業となった。
2K Sports ; 2K Sportsとは、株式会社テイクツー・インタラクティブソフトウェアの子会社である。この企業は一つの開発スタジオ、2005年にセガから買収したVisual Conceptsからなりたっている。2K SportsはおもにNBA 2Kシリーズのようなアメリカ合衆国のスポーツのコンピュータゲームを開発、発売している。
343 Industries ; 343 Industr

次は、東京都の情報を取得します。

In [ ]:
from SPARQLWrapper import SPARQLWrapper

#東京都
sparql4 = SPARQLWrapper(endpoint='http://ja.dbpedia.org/sparql', returnFormat='json')
sparql4.setQuery("""
    PREFIX dbpedia-owl:  <http://dbpedia.org/ontology/>
    SELECT DISTINCT * WHERE {
  <http://ja.dbpedia.org/resource/東京都> ?p ?o .
}
""")
results4 = sparql4.query().convert()
print(results4)

{'head': {'link': [], 'vars': ['p', 'o']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#Thing'}}, {'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://www.w3.org/2003/01/geo/wgs84_pos#SpatialThing'}}, {'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://dbpedia.org/ontology/AdministrativeRegion'}}, {'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://dbpedia.org/ontology/Region'}}, {'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://schema.org/AdministrativeArea'}}, {'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type':

次は、全国ゆるキャラの情報を取得します。100個に限定する。

In [ ]:
from SPARQLWrapper import SPARQLWrapper

#全国ゆるキャラ
sparql5 = SPARQLWrapper(endpoint='http://ja.dbpedia.org/sparql', returnFormat='json')
sparql5.setQuery("""
    PREFIX dbpedia-owl:  <http://dbpedia.org/ontology/>
    SELECT DISTINCT ?mascot ?name WHERE {
    ?areamascots skos:broader <http://ja.dbpedia.org/resource/Category:地域限定のマスコット> ;
    rdfs:label ?areaname .
    ?mascot dbo:wikiPageWikiLink ?areamascots ;
    rdfs:label ?name.
} LIMIT 100
""")
results5 = sparql5.query().convert()
print(results5)

{'head': {'link': [], 'vars': ['mascot', 'name']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'mascot': {'type': 'uri', 'value': 'http://ja.dbpedia.org/resource/カボたん'}, 'name': {'type': 'literal', 'xml:lang': 'ja', 'value': 'カボたん'}}, {'mascot': {'type': 'uri', 'value': 'http://ja.dbpedia.org/resource/キュータ'}, 'name': {'type': 'literal', 'xml:lang': 'ja', 'value': 'キュータ'}}, {'mascot': {'type': 'uri', 'value': 'http://ja.dbpedia.org/resource/クウタン'}, 'name': {'type': 'literal', 'xml:lang': 'ja', 'value': 'クウタン'}}, {'mascot': {'type': 'uri', 'value': 'http://ja.dbpedia.org/resource/コトノちゃん'}, 'name': {'type': 'literal', 'xml:lang': 'ja', 'value': 'コトノちゃん'}}, {'mascot': {'type': 'uri', 'value': 'http://ja.dbpedia.org/resource/ザ・セサミブラザーズ'}, 'name': {'type': 'literal', 'xml:lang': 'ja', 'value': 'ザ・セサミブラザーズ'}}, {'mascot': {'type': 'uri', 'value': 'http://ja.dbpedia.org/resource/シモンちゃん'}, 'name': {'type': 'literal', 'xml:lang': 'ja', 'value': 'シモンちゃん'}}, {'mascot': {'type': 'u

取得した情報から、ゆるキャラの名前一覧を印刷します。

In [ ]:
#結果を整形しPrint する
for result in results5["results"]["bindings"]:
    print(result["name"]["value"])

カボたん
キュータ
クウタン
コトノちゃん
ザ・セサミブラザーズ
シモンちゃん
スギッチ
タボくん
チカパパ
トラムクン
ニャオざね
ハリー・ホーク
バスママ
ブルーダル
ポッポくん
マーくん
ラッキー (阪神タイガース)
レルヒさん
京町セイカ
原付萌奈美
喜楽総帥
四国萌ぇ隊
志摩ちゃん
札幌大通公園きびっち
801ちゃん
AGC38
Baby☆B
GJ8マン
GO太くん
JUNI
Juratic
Mr.カラスコ
OH!くん
Onちゃん
Onちゃんの仲間
UPるんチャン
あいのすけ
あかんたれのすけ
あさっぴー
あざみ
あしかがひめたま
あななん
あぶらげんしん
あまてるこ
あみたん娘
あら坊
あるぽ
あわみちゃん
あんてなちゃん
いかずきんズ
いしおか恋瀬姫
いしきりん
いしだみつにゃん
いなりん
いなッピー
いまいち萌えない娘
いまりんモーモちゃん
いもメン
いやしす
うぉーたん
うきぴー
うさBちゃん
うずしおくん
うずぴー
うだつまる
うどん脳
うないさん
うながっぱ
うなりくん
うにまる
えびー太くん
えんむちゃん
おいちごちゃん
おおふなトン
おおまぴょん
おこみん
おたる運がっぱ
おっ!サン
おのくん
おばこ娘
おぶせくりちゃん
おぶちゃん
おもぞう
お猿のくぅ
かっ・ちゅうくん
かなかなかぞく
かながわキンタロウ
かのやカンパチロウ
かほピョン
かみたん
からす天狗うじゅ
かわぐっち
かわさきノルフィン
かわたな戦隊クジャクマン
かわたな戦隊クジャクマンZ
かんザラシ
がくとくん
がんす娘。
きくちくん
きさポン


最後に新宿区の情報を取得する。

In [ ]:
from SPARQLWrapper import SPARQLWrapper

#
sparql6 = SPARQLWrapper(endpoint='https://data.e-stat.go.jp/lod/sparql/alldata/query', returnFormat='json')
sparql6.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sdmx-dimension: <http://purl.org/linked-data/sdmx/2009/dimension#>
PREFIX estat-measure: <http://data.e-stat.go.jp/lod/ontology/measure/>
PREFIX cd-dimension: <http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/>
PREFIX cd-code: <http://data.e-stat.go.jp/lod/ontology/crossDomain/code/>
PREFIX g00200521-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/g00200521/dimension/2010/>
PREFIX g00200521-code-2010:<http://data.e-stat.go.jp/lod/ontology/g00200521/code/2010/>
select  ?year ?population
where {
      ?s estat-measure:population ?population ;
         sdmx-dimension:refArea / rdfs:label "新宿区"@ja ;
         cd-dimension:timePeriod ?year ;
         cd-dimension:sex cd-code:sex-all ;
         cd-dimension:nationality cd-code:nationality-japan ;
         g00200521-dimension-2010:area g00200521-code-2010:area-all ;
         cd-dimension:age cd-code:age-all .
}
""")
results6 = sparql6.query().convert()
print(results6)

{'head': {'vars': ['YEAR', 'POPULATION'], 'links': []}, 'results': {'bindings': [{'YEAR': {'type': 'literal', 'value': '2015', 'datatype': 'http://www.w3.org/2001/XMLSchema#gYear'}, 'POPULATION': {'type': 'literal', 'value': '300013', 'datatype': 'http://www.w3.org/2001/XMLSchema#decimal'}}, {'YEAR': {'type': 'literal', 'value': '2010', 'datatype': 'http://www.w3.org/2001/XMLSchema#gYear'}, 'POPULATION': {'type': 'literal', 'value': '286398', 'datatype': 'http://www.w3.org/2001/XMLSchema#decimal'}}]}}


取得した新宿区の情報から、人口を取得してプリントする。

In [ ]:
#結果を整形しPrint する
for result in results6["results"]["bindings"]:
    print(result["YEAR"]["value"],";",result["POPULATION"]["value"])

2015 ; 300013
2010 ; 286398
